# Day 11 - Dumbo Octopus

https://adventofcode.com/2021/day/11

In [13]:
from pathlib import Path

INPUTS = Path("input.txt").read_text().strip().split("\n")
# Separate the strings into a list of integers for easier processing later
INPUTS = [[int(y) for y in x] for x in INPUTS]


## Part 1

In [14]:
from copy import deepcopy


def flash(
    grid: list[list[int]],
    x: int,
    y: int,
) -> tuple[list[list[int]], int]:
    """Cause a flash at coord (x,y).

    Returns a tuple of (grid, num_flashes),
    with the new grid layout post-flash and the number of flashes caused
    by this function.
    """
    num_flashes = 1
    new_grid = deepcopy(grid)
    new_grid[x][y] = 0
    affected = []

    # Upper row
    if x > 0:
        if y > 0:
            affected.append((x - 1, y - 1))
        affected.append((x - 1, y))
        if y < len(grid[x]) - 1:
            affected.append((x - 1, y + 1))

    # Middle row
    if y > 0:
        affected.append((x, y - 1))
    if y < len(grid[x]) - 1:
        affected.append((x, y + 1))

    # Lower row
    if x < len(grid) - 1:
        if y > 0:
            affected.append((x + 1, y - 1))
        affected.append((x + 1, y))
        if y < len(grid[x]) - 1:
            affected.append((x + 1, y + 1))

    for point_x, point_y in affected:
        if new_grid[point_x][point_y] > 0:
            new_grid[point_x][point_y] += 1
            if new_grid[point_x][point_y] > 9:
                new_grid, flashes = flash(grid=new_grid, x=point_x, y=point_y)
                num_flashes += flashes
    return new_grid, num_flashes


def flash_cycle(grid: list[list[int]]) -> tuple[list[list[int]], int]:
    num_flashes = 0

    # 1: increase all energy levels by 1
    new_grid = deepcopy(grid)
    for x, row in enumerate(new_grid):
        for y, _ in enumerate(row):
            new_grid[x][y] += 1

    # 2: cause flashes
    for x, row in enumerate(new_grid):
        for y, _ in enumerate(row):
            if new_grid[x][y] > 9:
                new_grid, flashes = flash(grid=new_grid, x=x, y=y)
                num_flashes += flashes

    # return the new grid and number of flashes caused
    return new_grid, num_flashes


Some testing against the examples on AoC site:

In [15]:
def test_flash_cycle():
    grid = [
        [1, 1, 1, 1, 1],
        [1, 9, 9, 9, 1],
        [1, 9, 1, 9, 1],
        [1, 9, 9, 9, 1],
        [1, 1, 1, 1, 1],
    ]
    new_grid, flashes = flash_cycle(grid=grid)
    expected = [
        [3, 4, 5, 4, 3],
        [4, 0, 0, 0, 4],
        [5, 0, 0, 0, 5],
        [4, 0, 0, 0, 4],
        [3, 4, 5, 4, 3],
    ]
    assert expected == new_grid
    assert flashes == 9


def test_big_flash_cycles():
    start = [
        [5,4,8,3,1,4,3,2,2,3],
        [2,7,4,5,8,5,4,7,1,1],
        [5,2,6,4,5,5,6,1,7,3],
        [6,1,4,1,3,3,6,1,4,6],
        [6,3,5,7,3,8,5,4,7,8],
        [4,1,6,7,5,2,4,6,4,5],
        [2,1,7,6,8,4,1,7,2,1],
        [6,8,8,2,8,8,1,1,3,4],
        [4,8,4,6,8,4,8,5,5,4],
        [5,2,8,3,7,5,1,5,2,6],
    ]
    grid = deepcopy(start)
    total_flashes = 0
    for _ in range(10):
        grid, flashes = flash_cycle(grid=grid)
        total_flashes += flashes
    assert total_flashes == 204


test_flash_cycle()
test_big_flash_cycles()

Now it's just a matter of running through the flash cycles the requisite number of times and summing our total flashes.

In [16]:
grid = deepcopy(INPUTS)
total_flashes = 0
for _ in range(100):
    grid, flashes = flash_cycle(grid=grid)
    total_flashes += flashes

print(f"Total flashes after 100 cycles: {total_flashes}")

Total flashes after 100 cycles: 1713


## Part 2

Since we already have the method to return a new grid of octopuses set up, we can change from a set number of iterations to an arbitrary number using the dreaded `while` loop!

Just need a little helper function to determine when the octopuses are all synced up to act as a test and we're good:

In [17]:
def all_synced(grid: list[list[int]]) -> bool:
    flattened = [x for y in grid for x in y]
    return len(set(flattened)) == 1


Certainly I could make the test more robust, checking that all are 0 or some such, but it's not really needed here. If they're all synced, they're all the same number. That means the `set()` generated from the flattened list of all grid elements will be a single element long, the lone `0`.

With that, the solution just relies on grabbing the new grid, testing it each cycle, and `break`ing when we find the sync state. The `cycle` counter then gives us the step number we need.

In [18]:
cycle = 0
sync_grid = deepcopy(INPUTS)
while True:
    cycle += 1
    sync_grid, _ = flash_cycle(grid=sync_grid)
    if all_synced(grid=sync_grid):
        break

print(f"First synced step: {cycle}")


First synced step: 502
